In [ ]:
import sys
sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/src")
sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/notebooks/anu_winter_2024")
#sys.path.append("/Users/anuram/Documents/Documents/hydrogenase-ftir/src/hydrogenase_processing")

In [ ]:
%load_ext autoreload
%autoreload 2

#Setting Up and Importing the Necessary Packages/Libraries
##Package for reading in Bruker OPUS type files
from brukeropusreader import read_file
import matplotlib.pyplot as plt
from scipy.interpolate import UnivariateSpline
import pathlib
import numpy as np
import pandas as pd
#Local Functions
from hydrogenase_processing.cut_range import cut_range_subtract_multiple_wv
from hydrogenase_processing.second_deriv import second_deriv, second_deriv_batch
from hydrogenase_processing.anchor_points import interact
from hydrogenase_processing.anchor_points import baseline_spline, get_peaks_absorbance, baseline_correction, get_baseline_peak_index, get_peak_wid_at_half_height
from hydrogenase_processing.vaporfit import atm_subtraction
from hydrogenase_processing.vaporfit import AtmFitParams
from scipy.signal import find_peaks, peak_widths
#from hydrogenase_processing.anchor_points import get_peaks, get_start_end_anchorpoints, get_all_anchor_points, baseline_spline, get_peaks_absorbance, baseline_correction, get_peak_baseline_absorbance, plot_baseline_data
from hydrogenase_processing.peak_fit import gaussian, peak_fit, lorentzian
from hydrogenase_processing.io import import_run_data


# Importing spectra data and water vapor data

In [ ]:
#Paths to Local Data
path_to_water_vapor_data = pathlib.Path("../../data/opus_files/water_vapor")
path_to_output_plots_= pathlib.Path("../../data/output_plots/")
path_to_pD6_data = pathlib.Path("../../data/opus_files/pD6")

In [ ]:
pD6_raw_data = import_run_data(path_to_pD6_data)
wv_data = import_run_data(path_to_water_vapor_data)

# Subtracting water vapor from raw spectra

In [ ]:
#Initializing dict of post water vapor subtraction spectra
cut_range_sub_wv_data = cut_range_subtract_multiple_wv(pD6_raw_data, wv_data, range_start = 2150, range_end = 1850)


## Plot the subtracted data for visual inspection

In [ ]:
sample_raw = cut_range_sub_wv_data["Hyd1 pD6 ACT.0008"]
sample_raw[0][0].plot()

print(sample_raw[0][0].fit_atm_params)

## Get the second derivatives

In [ ]:
second_deriv_pD6_data = second_deriv_batch(cut_range_sub_wv_data)

# Baseline correction and cruve fit for an example spectra workflow:

In [ ]:
sample_second_deriv = second_deriv_pD6_data["Hyd1 pD6 ACT.0005_second_deriv"]
#print(sample_second_deriv)

## Using a user interface please select the appropriate number of peaks using the threshold variable and number of anchor points using adjustment factor variable

In [ ]:
#anchor_point_dict, deriv_x_peak_val, anchor_points_raw_data, y_corr_abs
anchor_point_dict, deriv_x_peak_val, anchor_points_raw_data, y_corr_abs = interact(sample_second_deriv, sample_raw, 0.4, 1.8)

## Fit baseline spline through these anchor points to be used for baseline correction of raw spectra

In [ ]:
baseline_curve = baseline_spline(anchor_point_dict, smooth=10**-9)
peak_wv, peak_abs = get_peaks_absorbance(deriv_x_peak_val, anchor_points_raw_data, y_corr_abs)

plt.plot(anchor_points_raw_data, y_corr_abs, label = 'raw data')
plt.plot(baseline_curve['wavenumber'], baseline_curve['absorbance'], 'r-', label = 'baseline spline')
plt.plot(peak_wv, peak_abs, 'go', label = 'peaks')
plt.ylabel('Absorbance')
plt.xlabel('wavenumber')
plt.legend()

## Subtract the baseline spline from the raw data and plot the results

In [ ]:
baseline_corrected_abs = baseline_correction(baseline_curve, anchor_points_raw_data, y_corr_abs)
peak_wv_index, peak_wv_baseline, peak_baseline_abs = get_baseline_peak_index(baseline_corrected_abs, anchor_points_raw_data, peak_wv)


# Get peak widths at half peak height

In [ ]:
peakwidth_at_half_height = get_peak_wid_at_half_height(baseline_corrected_abs, peak_wv_index)
print(peakwidth_at_half_height)

# Fitting gaussian and lorentizian curves on the baseline corrected data

In [ ]:
params, rmse = peak_fit('Gaussian', anchor_points_raw_data, baseline_corrected_abs, peak_wv_index)
print(f'Root mean squared error is {rmse}')

In [ ]:
params, rmse = peak_fit('Lorentzian', anchor_points_raw_data, baseline_corrected_abs, peak_wv_index)
print(f'Root mean squared error is {rmse}')